In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install rouge

In [ ]:
from __future__ import unicode_literals, print_function, division
%matplotlib inline
debug = True
to_print_epoch = True
from nltk.translate import bleu
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

from io import open
import unicodedata
import string
import re
import random
import os

# Pytorch Functionlaities

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import time
import math
from rouge import Rouge
import matplotlib.pyplot as plt

In [ ]:
MAX_LENGTH = 200
root_directory = "/kaggle/input/google-dataset/"
epoch = 1000
print_every_epoch = 5
no_of_hidden_size = 512
lcl_learning_rate = 0.01
dropout = 0
n_layers = 2
teacher_forcing_ratio = 0.5

Prepare Input Data For Training 

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not","didn't": "did not",
"doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
"he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
"I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
"i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
"mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
"mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
"oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
"she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
"should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
"this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
"they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
"wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
"we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
"what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
"where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
"why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"}

In [ ]:
def process_text(text,flag=False):
    stop_words = stopwords.words('english')
    text = text.lower()
    text = text.replace('\n','')
    text = re.sub(r'\(.*\)','',text)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text)
    text = re.sub(r'\.',' . ',text)
    text = text.replace('.','')
    text = text.split()
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    newtext = []
    for word in text:
        if word not in stop_words and len(word)>0:
            newtext.append(word)
    text = newtext
    if flag:
        text = text[::-1]
    text = " ".join(text)
    text = text.replace("'s",'') 
    return text

In [ ]:
def prepareInput(sourcefilePath,destFilePath,fileName):
    source = sourcefilePath
    target = destFilePath
    save_trans = fileName

    corpus_source = open(source, 'r').readlines()
    corpus_target = open(target, 'r').readlines()
  
    writer = open(save_trans, 'w')
    for k, v in zip(corpus_source, corpus_target):
        k = process_text(k,True)
        v = process_text(v,False)
        writer.write(k + '\t' + v + '\n')
    writer.flush()
    writer.close()

In [ ]:
SOS_token = 0
EOS_token = 1
UNK_token = 2
PAD_token = 3

class Lang:
    def __init__(self, name):
        self.name = name  
        self.word2index,self.word2count,self.index2word = {},{},{}
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS", UNK_token:"UNK"}
        self.n_words = 3  # Count SOS and EOS and UNK

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
def parseLanguageInput(lang1,lang2,fullFilePath):

    lines = open(fullFilePath, encoding='utf-8').read().strip().split('\n')
    pairs = [[s for s in l.split('\t')] for l in lines]

    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

def prepareData(lang1,lang2,fullFilePath):
    input_lang, output_lang, pairs = None,None,None
    input_lang, output_lang, pairs = parseLanguageInput(lang1,lang2,fullFilePath)

    for pair in pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    return input_lang, output_lang, pairs

Designing Encoder

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers = n_layers, bidirectional=True)

    def forward(self, input, hidden):
#         print(input.size())
#         print(hidden.size())
        embedded = self.embedding(input).view(1, 1, -1)
#         print(embedded.size())
        output = embedded#.unsqueeze(0)
#         print(output.size())
        
        output, hidden = self.gru(output, hidden)
#         print("Encoder: ",output.size(),hidden.size())
        return output, hidden

    def initHidden(self):
        return torch.zeros(n_layers*2, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size , num_layers = n_layers, bidirectional=True)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
#         print("Decoder: ",attn_weights.size(),attn_weights.unsqueeze(0).size() )
#         print("Decoder: ",encoder_outputs.size(), encoder_outputs.unsqueeze(0).size())
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))
#         print("Decoder: ",attn_applied.size())
#         print("Decoder: ",embedded.size())
#         output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = attn_applied[0]
        
#         print("Decoder: ",output.size())
        output = self.attn_combine(output)
#         print("Decoder: ",output.size())
        output = output.unsqueeze(0)
#         print("Decoder: ",output.size())
        output = F.relu(output)
#         print("Decoder: ",output.size())
#         print("Decoder: ",hidden.size())
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device)

Helper Methods -----
Prepare Training Data to put EndOfMarker at end of the Sentence

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] if word in lang.word2index else UNK_token for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Train

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    no_of_epoch = []
    print_loss_total = 0 
    plot_loss_total = 0

    encoder_optimizer = optim.ASGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.ASGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        
        training_pair = training_pairs[iter - 1]
        input_tensor,target_tensor = training_pair[0],training_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder,decoder, encoder_optimizer, decoder_optimizer, criterion)

        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            if to_print_epoch:
                  print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            no_of_epoch.append(iter)
            plot_loss_total = 0

    return plot_losses,no_of_epoch    

Evaluate the Model

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
#         print(sentence)
        input_tensor = tensorFromSentence(input_lang, sentence)
#         print(input_tensor.size())
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
prefix = "train"
sourceLangPath = root_directory +prefix+".original"
sourcePrefix = "original"
targetLangPath = root_directory+prefix+".compressed"
targetPrefix = "compressed"
fullFilePathForData = prefix + "_" + sourcePrefix + "_" + targetPrefix + ".txt"
teacher_forcing_ratio = 0.5

Set Up the Input Data 

In [ ]:
# create Custom File having both Langauges  
isFilePresent = os.path.isfile(fullFilePathForData)
# if isFilePresent == False:
print("New File Created")
# make_connection()
prepareInput(sourceLangPath,targetLangPath,fullFilePathForData)
# else:
#   print("File is not created Again")

input_lang, output_lang, pairs = prepareData(sourcePrefix, targetPrefix,fullFilePathForData)


In [ ]:
print(pairs[0])
print(pairs[1])
print(pairs[2])

In [ ]:
def load_saved_encoder(lcl_input_lang,lcl_hidden_size,lcl_encoder_model_path):
    device = torch.device('cpu')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    encoder_model = EncoderRNN(lcl_input_lang.n_words, lcl_hidden_size).to(device)
    encoder_model.load_state_dict(torch.load(lcl_encoder_model_path, map_location=device))
    return encoder_model

def load_saved_decoder(lcl_hidden_size,lcl_output_lang,lcl_decoder_model_path):
    # device = torch.device('cpu')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    decoder_model = AttnDecoderRNN(lcl_hidden_size,lcl_output_lang.n_words,dropout,MAX_LENGTH).to(device)
    decoder_model.load_state_dict(torch.load(lcl_decoder_model_path, map_location=device))
    return decoder_model

def load_obj(obj_type,obj_name_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    obj_type = torch.load(obj_name_path, map_location=device)
    return obj_type
  
encoder_model_path = "Encoder_Model.pt"
decoder_model_path = "Decoder_Model.pt"
params = "params.pt"
vocab_params = "vocab.pt"

train_result_data_path = "train_Result_Epoch_"+ str(epoch) +"_Hid_size_"+ str(no_of_hidden_size) +".pt"
test_result_data_path = "test_Result_Epoch_"+ str(epoch) +"_Hid_size_"+ str(no_of_hidden_size) + ".pt"   
encoder_model_path = "Ep_"+ str(epoch) +"_Hd_"+ str(no_of_hidden_size) + "_lr_"+ str(lcl_learning_rate) +"_"+ encoder_model_path   
decoder_model_path = "Ep_"+ str(epoch) +"_Hd_"+ str(no_of_hidden_size) + "_lr_"+ str(lcl_learning_rate) +"_"+ decoder_model_path 
vocab_params = "Ep_"+ str(epoch) +"_Hd_"+ str(no_of_hidden_size) + "_lr_"+ str(lcl_learning_rate) +"_"+ vocab_params 

print("Encoder Model Path :" ,encoder_model_path)
print("Decoder Model Path :" ,decoder_model_path)
print("params Path :" ,params)
print("train file Path :",train_result_data_path)
print("test file Path :",test_result_data_path)
print("Encoder Model Exist " ,os.path.isfile(encoder_model_path))
print("Decoder Model Exist " ,os.path.isfile(decoder_model_path))

Invoke Seq2Seq Model

In [ ]:
encoder_model = None
decoder_model = None
model_performance = {}

# if os.path.isfile(encoder_model_path) == False or os.path.isfile(decoder_model_path) == False:
hidden_size = no_of_hidden_size
encoder_model = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_model = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=dropout).to(device)
plot_losses,no_of_epoch = trainIters(encoder_model, decoder_model, epoch, print_every=print_every_epoch,learning_rate=lcl_learning_rate)
folder_path = F""+root_directory+"{encoder_model_path}"
torch.save(encoder_model.state_dict(), encoder_model_path)
folder_path = F""+root_directory+"{decoder_model_path}"
torch.save(decoder_model.state_dict(), decoder_model_path)

lcl_params = {'loss_list':plot_losses,'epoch_list':no_of_epoch,'lr':lcl_learning_rate,"dropout":dropout, "MAX_LENGTH":MAX_LENGTH,"epoch":epoch, "no_of_hidden_size":no_of_hidden_size}
lcl_params["lcl_learning_rate"] = lcl_learning_rate

vocab = {'input_lang':input_lang, 'output_lang':output_lang}
model_performance = lcl_params
torch.save(lcl_params, params)
torch.save(vocab, vocab_params)
# else:
#     print("Model Already Exist")
#     hidden_size = no_of_hidden_size
#     encoder_model = load_saved_encoder(input_lang,hidden_size,encoder_model_path)
#     decoder_model = load_saved_decoder(hidden_size,output_lang,decoder_model_path)
#     model_performance = load_obj(model_performance,params)
#     vocab = load_obj(vocab,vocab_params)
print(model_performance)
print(vocab)
print(encoder_model)
print(decoder_model)

In [ ]:
plt.plot(plot_losses)
plt.show()

Assessment of Model

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction    

def calculate_rouge(rouge, pred_trg, real_trg):

    pred_trg = " ".join(pred_trg)
    real_trg = " ".join(real_trg[0])
    if len(pred_trg) > len(real_trg):
        diff = len(pred_trg) - len(real_trg)
        real_trg = real_trg +" "+  "#"*(diff-1)
    elif len(pred_trg) < len(real_trg):
        diff = len(real_trg) - len(pred_trg)
        pred_trg = pred_trg +" "+ "#"*(diff-1)
    scores = rouge.get_scores(pred_trg, real_trg)
    return scores 

def calculate_Result(encoder, decoder,lcl_pairs, n=50):
    result_value_rouge_score = []
    rouge = Rouge()
    
    for i in range(n):
        pair = random.choice(lcl_pairs)

        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)

        reference = [pair[1].split()]

        output_words = output_words[:-1]
        target_predicted = output_words
        
        score = calculate_rouge(rouge,target_predicted,reference)
        result_value_rouge_score.append((pair[0],pair[1].split(),target_predicted,score))

    return result_value_rouge_score

Testing on Trained data

In [ ]:
result_value_rouge_score = calculate_Result(encoder_model, decoder_model,pairs)
result_value_rouge_score_dict = {}
result_value_rouge_score_dict['result'] = result_value_rouge_score 
torch.save(result_value_rouge_score_dict, train_result_data_path)
for item in result_value_rouge_score:
  print(" Source Language ",item[0])
  print(" Input Target",item[1])
  print(" Output Target",item[2])
  print(" Score ",item[3])

Testing on Test Data

In [ ]:
prefix = "test"
sourceLangPath = root_directory +prefix+".original"
sourcePrefix = "original"
targetLangPath = root_directory+prefix+".compressed"
targetPrefix = "compressed"
fullFilePathForData = prefix + "_" + sourcePrefix + "_" + targetPrefix + ".txt"

In [ ]:
# create Custom File having both Langauges  
isFilePresent = os.path.isfile(fullFilePathForData)
# if isFilePresent == False:
print("New File Created")
# make_connection()
prepareInput(sourceLangPath,targetLangPath,fullFilePathForData)
# else:
#   print("File is not created Again")

test_input_lang, test_output_lang, test_pairs = prepareData(sourcePrefix, targetPrefix,fullFilePathForData)
print(random.choice(test_pairs))

In [ ]:
result_value_rouge_score_test = calculate_Result(encoder_model, decoder_model,test_pairs)
result_value_rouge_score_dict_test = {}
result_value_rouge_score_dict_test['result'] = result_value_rouge_score_test 
torch.save(result_value_rouge_score_dict_test, test_result_data_path)
for item in result_value_rouge_score_test:
    print(" Source Language ",item[0])
    print(" Input Target",item[1])
    print(" Output Target",item[2])
    print(" Score ",item[3])